## Unbalanced power flow calculation with pandapower

We will create a simple three bus system to demonstrate how to run unbalanced power flow calculations with pandapower. First, we create the three buses as follows:

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_ext_grid,
    create_transformer_from_parameters,
    create_line_from_parameters,
    create_asymmetric_load
)
from pandapower.pf.runpp_3ph import runpp_3ph

In [ ]:
net = create_empty_network()
b1  = create_bus(net, 20.0)
b2  = create_bus(net, 0.4)
b3  = create_bus(net, 0.4)

The external grid connection can then be created with the short-circuit parameters s_sc_max_mva and rx_max as well as the parameters x0x_max and r0x0_max that define the zero sequence impedances. The ext_grid parameters are all defined in the [ext_grid parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/ext_grid.html#input-parameters).

In [ ]:
create_ext_grid(net, b1, s_sc_max_mva=1000, rx_max=0.1, x0x_max=1.0,
                   r0x0_max=0.1)

The transformer requires the additional parameters vk0_percent, vkr0_percent, mag0_percent, mag0_rx, vector_group and si0_hv_partial for unbalanced calculation. The transformer parameters are all defined in the [transformer parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/trafo.html#input-parameters).

In [ ]:
create_transformer_from_parameters(net, b1, b2, sn_mva=0.63,
                                      vn_hv_kv=20., vn_lv_kv=0.4,
                                      vkr_percent=0.1, vk_percent=6,
                                      vk0_percent=6, vkr0_percent=0.78125,
                                      mag0_percent=10000, mag0_rx=0.,
                                      pfe_kw=0.1, i0_percent=0.1,
                                      vector_group="Dyn", shift_degree=150,
                                      si0_hv_partial=0.9)

For lines, the zero sequence impedances r0_ohm_per_km, x0_ohm_per_km and c0_nf_per_km are required in addition to the balanced parameters. The line parameters are all defined in the [line parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/line.html#input-parameters).

In [ ]:
create_line_from_parameters(net, b2, b3, length_km=0.1, r0_ohm_per_km=0.0848,
                               x0_ohm_per_km=0.4649556, c0_nf_per_km=230.6,
                               max_i_ka=0.963, r_ohm_per_km=0.0212,
                               x_ohm_per_km=0.1162389, c_nf_per_km= 230)

Finally, to create an unbalanced power flow, we create an asymmetric load. The input parameters are all defined in the [asymmetric load parameter documentation](https://pandapower.readthedocs.io/en/develop/elements/asymmetric_load.html#input-parameters).

In [ ]:
create_asymmetric_load(net, b3, p_a_mw=0.25, p_b_mw=0.18, p_c_mw=0.20, type="wye")

We can now run an unbalanced power flow:

In [ ]:
runpp_3ph(net)

The results are stored in the result tables with suffix "_3ph", such as "res_bus_3ph", "res_line_3ph" etc.:

In [ ]:
net.res_bus_3ph

We can check the voltage in all phases as well as the unbalance percentage according to IEC 62749:

In [ ]:
net.res_bus_3ph[["vm_a_pu", "vm_b_pu", "vm_c_pu", "unbalance_percent"]]

The line and trafo results provide currents and loadings in different phases or overall loading, which is defined as the maximum of all phase loadings.

In [ ]:
net.res_line_3ph[["i_a_from_ka", "i_b_to_ka", "loading_b_percent", "loading_percent"]]

In [ ]:
net.res_trafo_3ph[["i_a_hv_ka", "i_b_lv_ka", "loading_b_percent", "loading_percent"]]